# Spelling correction

In [2]:
import json
import nltk
import spacy
import os
import pandas as pd
from collections import defaultdict

## Run a statistical language model

In [1]:
import operator
import numpy as np
from collections import defaultdict

class LM(object):
    
    def __init__(self, corpus, ngram_size=2, bos='+', eos='#', k=1):
        
        """
        :param corpus (list): a list of sentences, where each sentence is a list of tokens. 
        :param ngram_size (integer): specifies the size of the n-grams used for language modeling (default=2). 
        :param k (float or integer): smoothing value used when computing n-gram probabilities (default=1).
        :param bos (string): the Beginning-of-Sentence symbol (default='+').
        :param eos (string): the End-of-Sentence symbol (default='#').
        
        This class creates an LM object with attributes: 
            - k (float or integer): smoothing value. 
            - ngram_size (integer): size of the n-grams. 
            - bos (string): beginning-of-sentence symbol.  
            - eos (string): end-of-sentence symbol. 
            - corpus (nested list): a list of lists of strings representing the corpus.
            - vocab (set): tokens derived from the corpus. 
            - vocab_size (integer): size of the vocabulary. 
            - counts (default dict): stores n-gram counts. 
        """
        
        self.k = k
        self.ngram_size = ngram_size
        self.bos = bos
        self.eos = eos
        self.corpus = corpus
        self.vocab = self.get_vocab()
        self.vocab.add(self.eos)
        self.vocab_size = len(self.vocab)
        
    def get_vocab(self):
        
        """
        :return: a set of unique tokens in the corpus.
        
        This method derives the vocabulary from the training corpus (stored as a set of unique tokens). 
        """
        
        vocab = set()
        for sentence in self.corpus:
            for element in sentence:
                vocab.add(element)
        
        return vocab
                    
    def update_counts(self):
        
        """
        :return: None
        
        This method creates a defaultdict, padds the sentences with EoS and BoS, and updates the n-gram counts for the LM object. 
        """
        
        r = self.ngram_size - 1
        
        self.counts = defaultdict(dict)
        
        for sentence in self.corpus:
            s = [self.bos]*r + sentence + [self.eos]
            
            for idx in range(self.ngram_size-1, len(s)):
                ngram = self.get_ngram(s, idx)
                
                try:
                    self.counts[ngram[0]][ngram[1]] += 1
                except KeyError:
                    self.counts[ngram[0]][ngram[1]] = 1
                        
    def get_ngram(self, s, i):
        
        """
        :param s (list): tokens representing the sentence being processed.
        :param i (integer): index of the target token in the sentence.
        :return (tuple): the history and the target for the n-gram.
        
        This method extracts the n-gram for the current position (i) in the list of tokens (s),
        the n-gram consists of n-1 history tokens and 1 target token. 
        """
        
        ngram = s[i-(self.ngram_size-1):i+1]
        history = tuple(ngram[:-1])
        target = ngram[-1]
        return (history, target)
    
    def get_ngram_probability(self, history, target):
        
        """
        :param history (tuple): n-1 tokens representing the history of the n-gram.
        :param target (string): the target token of the n-gram, for which the probability is computed 
        :return (float): probability of the target given the observed history
        
        This method computes the probability of a target token given the history in the LM object.
        This probability is based on the smoothed counts of n-grams found in the corpus. 
        """
        
        try:
            ngram_tot = np.sum(list(self.counts[history].values())) + (self.vocab_size*self.k)
            try:
                transition_count = self.counts[history][target] + self.k
            except KeyError:
                transition_count = self.k
        except KeyError:
            transition_count = self.k
            ngram_tot = self.vocab_size*self.k
        
        return transition_count/ngram_tot 
    
    def perplexity(self, sentence):
        
        """
        :param sentence (list): tokens representing the sentence.
        :return (float): perplexity score of the sentence.
        
        This method computes the perplexity score of a given sentence using the formula pow(2.0, avg_entropy).
        This tells us how well the language model predicts the unseen sentence.
        """
        
        r = self.ngram_size - 1
        s = [self.bos]*r + sentence + [self.eos]
        

        probs = []
        for idx in range(self.ngram_size-1, len(s)):
            ngram = self.get_ngram(s, idx)
            probs.append(self.get_ngram_probability(ngram[0], ngram[1]))
                    
        entropy = np.log2(probs)
        avg_entropy = -1 * (sum(entropy) / len(entropy))
        return pow(2.0, avg_entropy)

## Load the data

In [3]:
# pre-processed corpus
with open('LM_trainingCorpus.json') as f:
    lm_trainingcorpus = json.load(f) # I did not keep the training corpus on a dataframe because of my computer's limited capacity 
# 15 sentences with typos
sentences_with_typos = pd.read_csv('sentence_with_typos.txt', sep='\t', header=0, names=['ID', 'Sentence'])
# from the SUBTLEXus lexicon
subtlexus = pd.read_csv('SUBTLEXus.txt', sep='\t', na_values=['NA'], low_memory=False)

# Selecting rows where 'Word' is not a string
non_string_rows = subtlexus[~subtlexus['Word'].apply(lambda x: isinstance(x, str))]

# Drop NaN rows and reset index
subtlexus = subtlexus.drop(index=non_string_rows.index[0]).reset_index(drop=True)

## Find which token contains a typo in the given sentences
Having a typo means that the word does not feature in SUBTLEXus. There is one and only one word containing a typo in each sentence, as defined in this way: the typo can result from the insertion/deletion/substitution of one or two characters. The generated .json file contains a simple dictionary mapping the sentence ID (as an integer) to the mistyped word (as a string).  

In [4]:
# find mistyped words in the input sentences

from nltk.tokenize import RegexpTokenizer
import re

# create list of lexicon tokens 
subtlexus_tok = list(subtlexus['Word'])

# function to find the nearest neighbor of a target  
def edit_distance(target):
    """
    This function takes a target word as input and calculates its minimum edit distance to other words in SubtlexUS.

    Args:
        target (str): target word for which minimum edit distance is to be calculated.

    Returns:
        tuple: A tuple containing the minimum edit distance and the list of words that have the same minimum edit distance.
    """
    # initialize variables
    min_edit = 100  # arbitrary large  
    neighbors = []
    
    # iterate over all the tokens in subtlexus_tok
    for word in subtlexus_tok:
        # if the word is a valid string
        if isinstance(word, str):
            # remove non-alphabetic characters
            word = re.findall(r'^[^a-zA-z]*([a-zA-Z]*)[^a-zA-z]*$', word)   
            if word and word[0] != target:
                # calculate edit distance
                d = nltk.edit_distance(word[0], target)
                if d < min_edit:
                    min_edit = d
                    neighbors = [word[0]]
                elif d == min_edit:
                    neighbors.append(word[0])
                else:
                    pass
    return (min_edit, neighbors)


def create_mistyped_dict():
    """
    Creates a dictionary of mistyped words from a dataframe of sentences with typos.
    
    Returns:
        mistyped_dict (dict): A dictionary where each key is a sentence ID and the value is a mistyped word.
    """
    # create a dictionary to store mistyped words
    mistyped_dict = {}  

    # loop through each sentence 
    for index, row in sentences_with_typos.iterrows():
        sentence_id = row['ID']
        sentence = row['Sentence']

        # tokenize sentence (ignoring special characters)
        tokenizer = RegexpTokenizer(r'\w+')
        words = tokenizer.tokenize(sentence)

        # loop through each word in the sentence
        for word in words:
            if word not in subtlexus_tok:
                print(word, 'is not in SubtlexUs')
                # find nearest neighbor 
                min_edit, neighbors = edit_distance(word)
                if min_edit <= 2:
                    mistyped_dict[sentence_id] = word
                    # only consider 1 mistyped word per sentence
                    break  
                    
    return mistyped_dict

mistyped_dict = create_mistyped_dict()

# store mistyped words in JSON file
with open('task2a_PaulaAraujoRabinovich_solution.json', 'w') as f:
    json.dump(mistyped_dict, f)

knoq is not in SubtlexUs
reserachers is not in SubtlexUs
grozn is not in SubtlexUs
quolg is not in SubtlexUs
waies is not in SubtlexUs
wintr is not in SubtlexUs
munors is not in SubtlexUs
surgicaly is not in SubtlexUs
aquire is not in SubtlexUs
acomodate is not in SubtlexUs
dats is not in SubtlexUs
collegue is not in SubtlexUs
layed is not in SubtlexUs
cate is not in SubtlexUs
giambic is not in SubtlexUs


## Find the words from SUBTLEXus with the smallest edit distance from each mistyped target
This should return the 3 words at the smallest edit distance, sorted by edit distance. However, if there are more words at the same edit distance than the third closest, the code should include all the words at the same edit distance. Therefore, supposing that the string 'abcdef' has two neighbors at edit distance 1, and four neighbors at edit distance 2, the third closest neighbor would be at edit distance 2, but there would be other three words at the same distance and then the code should thus return six neighbors for the target string. The generated .json file contains a dictionary mapping sentence IDs (as integers) to lists of tuples, where each tuple contains first the word (as a string) and then the edit distance (as an integer), with tuples sorted by edit distance in ascending order (smallest edit distance first). 

In [8]:
# retrieve the nearest neighbors based on edit distance

def find_best_neighbors(target):
    """
    Finds the 3 closest neighbors (or more, when applicable) to a given target word based on edit distance. 

    Args:
        target (str): The target word we want to find the closest neighbors for.

    Returns:
        dict: A dictionary containing the closest neighbors of the target word, with edit distances as keys and a list of closest neighbors as values.

    """
    
    neighbors = {}  # dictionary to store all neighbors 
    
    # for each word in the lexicon
    for word in subtlexus_tok:
        # if the word is a valid string
        if isinstance(word, str):
            # extract only alphabetic characters from the word
            word = re.findall(r'^[^a-zA-z]*([a-zA-Z]*)[^a-zA-z]*$', word)
            # check that the word (from subtlexus_tok) is not empty and that it is not the same as the target misspelled word 
            if word and word[0] != target:
                # calculate the distance between word in lexicon and target word
                d = nltk.edit_distance(word[0], target)
                # if distance is lower than our edit distance threshold
                if d <= 100:  
                    # if distance is not already in our dictionary of neighbors
                    if d not in neighbors:
                        # create new dictionary entry 
                        neighbors[d] = [word[0]]
                    else:
                        # extend an existing dictionary entry
                        neighbors[d].append(word[0])

    closest_neighbors = {} # dictionary to store only the closest neighbors
    
    # loop through each mistyped word, and store its closest neighbors
    for key, value in sorted(neighbors.items()):
        closest_neighbors[key] = value
        # if the length of the closest neighbors dictionary is already equal or greater than 3, break the loop
        if sum(len(v) for v in closest_neighbors.values()) >= 3:
            break  
    
    # the function returns the closest neighbors of one single target (one mistyped word)
    return closest_neighbors 


# this function to apply find_best_neighbors() to all 15 mistyped words
def get_result():

    """
    Applies find_best_neighbors() to all 15 mistyped words, and returns a dictionary.

    Returns:
    dict: A dictionary containing the closest neighbors of each mistyped word. The dictionary maps sentence IDs (as integers) to lists of tuples, 
    where each tuple contains first the word (as a string) and then the edit distance (as an integer).
    
    """
    
    results_dict = {}

    # iterate through each mistyped word (one per sentence)
    for key, mistyped_word in mistyped_dict.items():  
        
        # for each mistyped word, create a dictionary of its neighbors (keys being the edit-distance)
        closest_correct_words = find_best_neighbors(mistyped_word) 
        # example: {2: ['researchers'], 3: ['researcher', 'researches', 'reachers']}

        all_tuples = []  
        # here we will store the neighbors of the current mispelled word being processed in tuple form
        # example: [('researchers', 2), ('researcher', 3), ('researches', 3), ('reachers', 3)]

        # loop through dictionary (distance = dict key, neighbors = dict values) 
        for distance, neighbors in closest_correct_words.items(): 
            # for each potential neighbor of a given edit-distance
            for neighbor in neighbors: 
                # create a tuple with the (correct word, edit-distance)
                one_tuple = (neighbor, distance)
                # store the tuple in the list (there is one list of tuples for every mispelled word)
                all_tuples.append(one_tuple)
        
        results_dict[key] = all_tuples   

    return results_dict

edit_dist_dict = get_result()

# store candidates with smallest edit distance from each mistyped target in JSON file
with open('task3_PaulaAraujoRabinovich_solution.json', 'w') as f:
    json.dump(edit_dist_dict, f)
    
#print(edit_dist_dict)

## Best candidate according to frequency 
Then the list of candidate replacements found in the previous task can be used to find the best one according to candidate frequency (derived from SUBTLEXus) - if two or more candidates have the exact same frequency in SUBTLEX, then this progam chooses the one with the best edit distance. If two or more candidates at the same frequency also have the same edit distance, then it picks the one which comes first in alphabetical order. The program returns a .json file containing a simple dictionary mapping the sentence ID (as an integer) to a tuple containing the best candidate replacement (as a string) and its SUBTLEXus frequency value (as an integer).  

In [ ]:
# pick the best candidate according to SUBTLEXus frequency counts

def create_freq_dict(dictionary):
    """
    Creates a dictionary of frequencies for each mistyped word.
    
    Args:
        dictionary (dict): The dictionary of tuples (word, edit_distance) to process

    Returns:
    freq_dict (dict): A dictionary where the keys are the sentence ID and 
    the values are lists of tuples where each tuple contains a candidate word 
    and its frequency count in the SubtlexUs corpus.
    """
    freq_dict = {}
    
    for i in range(1,16):
        # retrieve candidate words for the current mistyped word being processed
        candidates = dictionary[i]
        # for each tuple in the candidate list
        for tup in candidates:
            # grab word
            word = tup[0]
            # find word frequency
            target_row = subtlexus[subtlexus['Word'] == word]
            freq = target_row['FREQcount'].values[0]
            # if row ID is not already in our dictionary of frequencies
            if i not in freq_dict:
                # create new dictionary entry 
                freq_dict[i] = [(word, freq)]
            else:
                # extend an existing dictionary entry
                freq_dict[i].append((word, freq))   
                
    return freq_dict

def find_tuple(string, dictionary):
    """
    Returns the tuple containing the input string from the given dictionary of tuples.

    Args:
        string (str): The string to search for in the tuples.
        dictionary (dict): The dictionary of tuples to search in.

    Returns:
        tuple: The tuple containing the input string in its first positional argument.
    """
    for value in dictionary.values():
        for tpl in value:
            if tpl[0] == string:
                return tpl
    return None


def get_best_word(i, freq_dict, edit_distance_dict):
    """
    Finds the tuple containing the ('best_word', frequency) for a sentence of index (i)
    One of 3 possible tuples can be returned:
        1) One with highest freq count
        2) One with highest freq count AND lowest edit distance (in case there are >1 words with highest freq count)
        3) One with highest freq count AND lowest edit distance AND highest in the alphabet is returned 
           (in case there are >1 words with highest freq count and lowest edit-dist)

    Args:
        i (integer): Sentence ID.
        freq_dict (dictionary): Words with frequency counts.
        edit_distance_dict (dictionary): Words with their edit-distance.

    Returns:
        tuple: ('best_word', frequency)
    """

    print(f'Processing {freq_dict[i]} for sentence {i}')
    # Determine the max value in the frequency dictionary
    max_value = max(freq_dict[i], key=lambda tpl: tpl[1])[1]
    print(f'\t Max frequency value is: {max_value} ')
    
    # Filter to find all tuples with the max value in terms of frequency counts
    max_tuples = [tpl for tpl in freq_dict[i] if tpl[1] == max_value] 
    print(f'\t Max tuple(s) is: {max_tuples} ')
    
    # fetch all words corresponding to the max frequencies
    maxfreq_words = [tpl[0] for tpl in max_tuples]
    print(f'\t Most frequent word(s) is: {maxfreq_words} ', end="\n")
    
    # IF there are multiple words with the same max frequency
    if len(max_tuples) > 1:
        print(f'\t There are multiple words with the same max frequency')
        # store relevant tuples (containing the words with max frequency)
        relevant_tuples = []
        for word in maxfreq_words:
            for tpl in edit_distance_dict[i]: # from the edit-distance dict
                if tpl[0] == word:
                    relevant_tuples.append(tpl)
                    
        # Determine the min value of these words in the edit-distance dictionary
        min_value = min(relevant_tuples, key=lambda tpl: tpl[1])[1]
        print(f'\t\t Minimum edit-distance value for word(s) with max frequency is: {min_value} ')
     
        # Filter to find all relevant_tuples with the minimum value in terms of the edit-distance
        min_tuples = [tpl for tpl in relevant_tuples if tpl[1] == min_value] 
        print(f'\t\t Minimum edit-distance tuple(s) for word(s) with max frequency is: {min_tuples} ')
        
        # fetch all words corresponding to the max frequencies and minimum edit-distance
        maxfreq_min_dist = [tpl[0] for tpl in min_tuples]
        
        # IF there are multiple words with the same max frequency AND minimum edit-distance
        if len(maxfreq_min_dist) > 1:
            print(f'\t\t There are multiple words with equal frequency counts and edit-distance: {min_tuples}')
            # Select tuple (from the list) that comes first in the alphabetical order
            alphabetical_max = min(min_tuples, key=lambda x: x[0])
            # alphabetical_max contains the tuple from the edit-distance dictionary -> (word, edit-distance)
            # we need the right tuple that contains the frequency counts -> (word, counts)
            
            # grab tuple with frequency counts 
            best_word = find_tuple(alphabetical_max[0], freq_dict)
            print(f'\t\t In this case, the best candidate is the one that comes first in the alphabetical order: {best_word}')
            return best_word  
        
        elif len(maxfreq_min_dist) == 1:
            # min_tuples[0]  contains the tuple from the edit-distance dictionary -> (word, edit-distance)
            # we need the right tuple that contains the frequency counts -> (word, counts)
            
            # grab tuple with frequency counts 
            best_word = find_tuple(min_tuples[0][0], freq_dict)
            return best_word
        
    elif len(max_tuples) == 1:
        # max_tuples[0] already contains the necessary information in the form of ('best_word', frequency)
        return max_tuples[0] #  
    
    else:
        print('max_tuples is of length 0')
        
def get_best_candidates(edit_dist_dict):
    """
    Finds the best candidate word for every one of the 15 mistyped words using frequency counts.

    Args:
        edit_dist_dict (dictionary): Dictionary with sentence IDs as dict keys, 
        and all candidate tuples as dict values, in the form of (word, edit-distance)

    Returns:
        dictionary: Dictionary with sentence IDs as keys, and the corresponding 
        best candidate tuple (word, frequency) as dict value.
         
    """
    
    best_candidates = {}
    # create dictionary to store frequency counts of words
    freq_dict = create_freq_dict(edit_dist_dict)
    # loop through each sentence to get its best word candidate
    for i in range(1,16):
        best_tuple = get_best_word(i, freq_dict, edit_dist_dict)
        # store the best candidate for sentence i
        best_candidates[i] = best_tuple
        
    return best_candidates

def convert(o):
    if isinstance(o, np.int64): return int(o)  
    raise TypeError
    
best_frequency_candidates = get_best_candidates(edit_dist_dict)

# store best candidates based on frequency in JSON file 
with open('task4_PaulaAraujoRabinovich_solution.json', 'w') as f:
    json.dump(best_frequency_candidates, f, default=convert)

## Best candidate according to perplexity

We can use the list of candidate replacements to find the best one according to its perplexity under a statistical language model of order 3 implemented using a Markov Chain with add-k smoothing (k=0.01) and estimated using the given corpus. If two or more candidates have the exact same perplexity in the input sentence, we can choose the one with the best edit distance. If two or more candidates at the same perplexity also have the same edit distance, then we can follow alphabetical order. The program returns a .json file containing a simple dictionary mapping the sentence ID (as an integer) to a tuple containing the best candidate replacement (as a string) and its perplexity under the specified language model (as a float). Not all candidate replacements might appear in the LM training corpus: in such casse the program won't map such candidate replacements to the UNK string though, or else the perplexity estimate would not reflect the specific candidate.

In [9]:
# pick the best candidate according to perplexity under the given language model

# create a language model based on tri-grams using smoothing constant k=0.01
trigram_model = LM(lm_trainingcorpus, ngram_size=3, bos='+', eos='#', k=0.01)
trigram_model.update_counts()

def create_variations():
    """
    Creates a dictionary with all possible sentence variations by replacing the mistyped words. 

    Returns:
        A dictionary where the keys are sentence IDs and the values are lists of all possible sentence variations.
    """
    all_sentences = list(sentences_with_typos['Sentence'])
    sentence_variations = {}
    
    # for each sentence ID and mistyped word  
    for sent_id, mistyped_word in mistyped_dict.items():
        # grab the original sentence string from the all_sentences list 
        original_sentence = all_sentences[sent_id-1]
        # grab list of neighbors for the sentence being processed
        neighbors = [elem[0] for elem in edit_dist_dict[sent_id]]
        
        # store only those neighbors that appear in our LM training vocab
        neighbors_invocab = []
        # loop through all neighbors
        for neighbor in neighbors:
            # only keep word candidates that appear in the LM training corpus 
            if neighbor in trigram_model.vocab:
                neighbors_invocab.append(neighbor)
                
        # create a variation of the original sentence by replacing the mistyped word with each neighbor in the LM vocabulary
        variations = [original_sentence.replace(mistyped_word, neighbor) for neighbor in neighbors_invocab]
        sentence_variations[sent_id] = variations

    return sentence_variations



def create_perplexity_dict(sentence_variations_dict):
    """
    Create a dictionary of perplexity scores for candidate replacements

    Parameters:
    dictionary (dict): a dictionary where keys are sentence IDs and values 
    are a list of variations for each sentence.

    Returns:
    dict: a dictionary where keys are sentence IDs and values are a tuple containing 
    the neighbor word involved in the perplexity calculation and its corresponding perplexity score.
    """    
    
    perplexity_dict = {} 
    # keys: sentence ID
    # values: (neighbor, perplexity)  
    
    # iterate through all main sentences 
    for i in range(1,16):
        
        # grab all sentence variations for the ith original sentence
        sentences = sentence_variations_dict[i]
        # store variations of sentence i (tokenized) in a list
        sentences_tokenized = []

        # tokenize sentence variations (for each sentence i being processed)
        for sentence in sentences:
            sentences_tokenized.append(nltk.wordpunct_tokenize(sentence))   

        # iterate through each sentence variation of sentence i
        for j in range(len(sentences_tokenized)):

            # identify the specific relevant neighbor involved in this particular sentence variation 
            all_neighbors = edit_dist_dict[i] # grab all neighbor words for sentence i  
            relevant_word = [t[0] for t in all_neighbors][j] # grab neighbor word j 

            # calculate perplexity for each sentence
            perplexity = trigram_model.perplexity(sentences_tokenized[j])

            # if row ID is not already in our dictionary of frequencies
            if i not in perplexity_dict:
                # create new dictionary entry 
                perplexity_dict[i] = [(relevant_word, perplexity)]
            else:
                # extend an existing dictionary entry
                perplexity_dict[i].append((relevant_word, perplexity))  
                
    return perplexity_dict
                
                


def get_best_word(i, perplexity_dict, edit_dist_dict):
    """
    Finds the tuple containing the ('best_word', perplexity) for a sentence of index (i)
    One of 3 possible tuples can be returned:
        1) One with lowest perplexity
        2) One with lowest perplexity AND lowest edit distance (in case there are >1 words with highest freq count)
        3) One with lowest perplexity AND lowest edit distance AND highest in the alphabet is returned 
           (in case there are >1 words with highest freq count and lowest edit-dist)

    Args:
        i (integer): Sentence ID.
        perplexity_dict (dictionary): Words with their perplexity.
        edit_distance_dict (dictionary): Words with their edit-distance.

    Returns:
        tuple: ('best_word', perplexity)
    """

    print(f'Processing {perplexity_dict[i]} for sentence {i}')
    # Determine the min value in the perplexity dictionary
    min_value = min(perplexity_dict[i], key=lambda tpl: tpl[1])[1]
    print(f'\t Min perplexity value is: {min_value} ')
    
    # Filter to find all tuples with the min value in terms of perplexity
    min_tuples = [tpl for tpl in perplexity_dict[i] if tpl[1] == min_value] 
    print(f'\t Min tuple(s) is: {min_tuples} ')
    
    # fetch all words corresponding to the min perplexity
    min_words = [tpl[0] for tpl in min_tuples]
    print(f'\t Lowest perplexity word(s) is: {min_words} ', end="\n")
    
    # IF there are multiple words with the same min perplexity
    if len(min_tuples) > 1:
        print(f'\t There are multiple words with the same min perplexity: {min_tuples}')
        # store relevant tuples (containing the words with min perplexity)
        relevant_tuples = []
        for word in min_words:
            for tpl in edit_dist_dict[i]: # from the edit-distance dict
                if tpl[0] == word:
                    relevant_tuples.append(tpl)
                    
        # Determine the min value of these words in the edit-distance dictionary
        min_dist_value = min(relevant_tuples, key=lambda tpl: tpl[1])[1]
        print(f'\t\t Minimum edit-distance value for word(s) with min perplexity is: {min_dist_value} ')
     
        # Filter to find all relevant_tuples with the minimum value in terms of the edit-distance
        min_dist_tuples = [tpl for tpl in relevant_tuples if tpl[1] == min_dist_value] 
        print(f'\t\t Minimum edit-distance tuple(s) for word(s) with min perplexity is: {min_dist_tuples} ')
        
        # fetch all words corresponding to the minimum perplexity and minimum edit-distance
        min_prp_min_dist = [tpl[0] for tpl in min_dist_tuples]
        
        # IF there are multiple words with the same minimum perplexity AND minimum edit-distance
        if len(min_prp_min_dist) > 1:
            print(f'\t\t There are multiple words with same min perplexity and edit-distance: {min_dist_tuples}')
            # Select tuple (from the list) that comes first in the alphabetical order
            alphabetical_max = min(min_dist_tuples, key=lambda x: x[0])
            print(f'\t\tt Keep the word that comes first in the alphabetical order: {alphabetical_max}')
            # alphabetical_max contains the tuple from the edit-distance dictionary -> (word, edit-distance)
            # we need the right tuple that contains the perplexity -> (word, perplexity)
            
            # grab tuple with perplexity
            best_word = find_tuple(alphabetical_max[0], perplexity_dict)
            return best_word  # 3
        
        elif len(min_prp_min_dist) == 1:
            # min_dist_tuples[0]  contains the tuple from the edit-distance dictionary -> (word, edit-distance)
            # we need the right tuple that contains the perplexity -> (word, perplexity)
            
            # grab tuple with perplexity
            best_word = find_tuple(min_dist_tuples[0][0], perplexity_dict)
            return best_word # 2
        
    elif len(min_tuples) == 1:
        # min_tuples[0] already contains the necessary information in the form of ('best_word', frequency)
        return min_tuples[0] # 1
    
    else:
        print('max_tuples is of length 0')
        
        
def get_best_candidates(edit_dist_dict):
    """
    Finds the best candidate word for every one of the 15 mistyped words using perplexity.

    Args:
        edit_dist_dict (dictionary): Dictionary with sentence IDs as dict keys, 
        and all candidate tuples as dict values, in the form of (word, edit-distance)

    Returns:
        dictionary: Dictionary with sentence IDs as keys, and the corresponding 
        best candidate tuple (word, frequency) as dict value.
         
    """
    # produce and store all sentence variations using candidate words
    sentence_variations = create_variations()
     # create dictionary to store perplexity scores of candidate words
    perplexity_dict = create_perplexity_dict(sentence_variations)

    best_candidates = {}

    # loop through each sentence to get its best word candidate
    for i in range(1,16):
        best_tuple = get_best_word(i, perplexity_dict, edit_dist_dict)
        # store the best candidate for sentence i
        best_candidates[i] = best_tuple
        
    return best_candidates

best_perplexity_candidates = get_best_candidates(edit_dist_dict)

# store best candidates based on perplexity in JSON file
with open('task5_PaulaAraujoRabinovich_solution.json', 'w') as f:
    json.dump(best_perplexity_candidates, f)

Processing [('know', 1002.7288999720108), ('knot', 5639.354069762032), ('knob', 8575.278397480908)] for sentence 1
	 Min perplexity value is: 1002.7288999720108 
	 Min tuple(s) is: [('know', 1002.7288999720108)] 
	 Lowest perplexity word(s) is: ['know'] 
Processing [('researchers', 14475.415924638553), ('researcher', 14476.248088113443), ('researches', 14475.415924638553)] for sentence 2
	 Min perplexity value is: 14475.415924638553 
	 Min tuple(s) is: [('researchers', 14475.415924638553), ('researches', 14475.415924638553)] 
	 Lowest perplexity word(s) is: ['researchers', 'researches'] 
	 There are multiple words with the same min perplexity: [('researchers', 14475.415924638553), ('researches', 14475.415924638553)]
		 Minimum edit-distance value for word(s) with min perplexity is: 2 
		 Minimum edit-distance tuple(s) for word(s) with min perplexity is: [('researchers', 2)] 
Processing [('grown', 1861.3599787842493), ('groin', 1858.1347148669051), ('groan', 1858.8593594720237)] for sen

## Conclusions

**Candidate replacements found when considering <u>frequency</u>:**
1. did you <font color='green'>know</font> that our fiends had a baby? 
2. I red that <font color='green'>researchers</font> managed to deviate the orbit of a comet with a satellite.
3. I could not help but <font color='red'>grown</font> in frustration when my computer trashed right before I finished my project.
4. I mate a <font color='red'>quote</font> from old clothes for my newborn nephew.
5. he <font color='green'>waves</font> his wand to get the attention of the waiter.
6. the roofs of the old house needed to he repaired before the <font color='red'>with</font>
7. <font color='orange'>minor</font> are not allowed to purchase cigarettes or alcohol.
8. the tumor was remove <font color='red'>strictly</font>
9. they <font color='red'>Squire</font> the company in order to expand their business.
10. the hotel was able to <font color='green'>accommodate</font> all of our needs.
11. I marked the <font color='green'>days</font> on my calendar so I would not forger.
12. I asked my <font color='orange'>college</font> for there opinion on the matter.
13. due to the economic situation, main employees were <font color='red'>played</font> off from their jobs.
14. I was refered to the specialist by my primary <font color='green'>care</font> physician.
15. many poets wrote in <font color='red'>gambit</font> pentameter.

*Outcome of frequency-based method:* There are 6 sentences out of 15 where a strictly accurate correction for the mistyped word was identified (40% accuracy).

**Candidate replacements found when considering <u>perplexity</u>:**
1. did you <font color='green'>know</font> that our fiends had a baby? 
2. I red that <font color='green'>researchers</font> managed to deviate the orbit of a comet with a satellite.
3. I could not help but <font color='red'>groin</font> in frustration when my computer trashed right before I finished my project.
4. I mate a <font color='red'>quote</font> from old clothes for my newborn nephew.
5. he <font color='green'>waves</font> his wand to get the attention of the waiter.
6. the roofs of the old house needed to he repaired before the <font color='red'>wind</font>
7. <font color='red'>jurors</font> are not allowed to purchase cigarettes or alcohol.
8. the tumor was remove <font color='red'>survival</font>
9. they <font color='red'>Squire</font> the company in order to expand their business.
10. the hotel was able to <font color='red'>accomodate</font> all of our needs.
11. I marked the <font color='green'>date</font> on my calendar so I would not forger.
12. I asked my <font color='orange'>colleague</font> for there opinion on the matter.
13. due to the economic situation, main employees were <font color='red'>Fayed</font> off from their jobs.
14. I was refered to the specialist by my primary <font color='green'>care</font> physician.
15. many poets wrote in <font color='green'>iambic</font> pentameter.

*Outcome of perplexity-based method:* There are 6 sentences out of 15 where a strictly accurate correction for the mistyped word was identified (40% accuracy).

**Which are better? Do they match what I consider to be the right replacement?**

In the section below, the corrected sentences are compared based on perplexity and frequency, with a focus only on the mistyped words of interest.

Sentences 1, 2, 5, and 14:
- Both approaches find the correct candidate replacements for these sentences, namely: "know", "researchers", "waves" and "care".

Sentence 3:
- Perplexity: groin
- Frequency: grown
- Both versions provide incorrect replacements for "grozn". The right correction should be "groan".

Sentence 4:
- Perplexity: quote
- Frequency: quote
- Both versions suggest "quote" as the replacement for "quolg," which is incorrect. The right correction should be "quilt".

Sentence 6:
- Perplexity: wind
- Frequency: with
- Both versions provide incorrect replacements for "wintr." The correct replacement should be "winter."

Sentence 7:
- Perplexity: jurors
- Frequency: minor
- The frequency-based version is better, as "minor" is closest to the word "minors", which is the correct replacement for "munors". The reason for the perplexity-based method's incorrect candidate selection for this sentence may be attributed to its analysis of the sentence's context. In this case, the language model might have associated the context with a legal scenario, causing it to predict "jurors" instead of "minor". The frequency-based method, on the other hand, does not rely on context and predicts based on the similarity of the misspelled word to the correct word, which led it to choose "minor".

Sentence 8:
- Perplexity: survival
- Frequency: strictly
- Both versions are incorrect; the right correction for "surgicaly" should be "surgically."

Sentence 9:
- Perplexity: Squire
- Frequency: Squire
- Both versions suggest "Squire" as the replacement for "aquire," which is incorrect. The right correction should be "acquire."

Sentence 10:
- Perplexity: accomodate
- Frequency: accommodate
- The frequency-based version is better, as "accommodate" is the correct replacement for "acomodate". This suggests that the frequency-based method may have had an advantage in this case, since the word "accomodate" is a common misspelling of "accommodate". Possibly, the language model could contain noise, inconsistencies, and/or misspellings. The presence of such errors in the training data might have influenced the predictions for candidate replacement, leading the program to choose the incorrect spelling in the perplexity-based method. 

Sentence 11:
- Perplexity: date
- Frequency: days
- The right correction for "dats" is ambiguous; it could be either "date" or "days." Both options are valid replacements.

Sentence 12:
- Perplexity: colleague
- Frequency: college
- Both versions seem somewhat incorrect. However, the perplexity-based version ("colleague") sounds slightly better, as it is likely that the author intended to write "colleagues" (in plural) in this sentence, given  the mispelled possessive pronoun "their" (typed as "there").

Sentence 13:
- Perplexity: Fayed
- Frequency: played
- Both versions are incorrect; the right correction for "layed" should be "laid."

Sentence 15:
- Perplexity: iambic
- Frequency: gambit
- The perplexity-based version is better, as "iambic" is the correct replacement for "gambit". This may be because the perplexity-based method employs a statistical language model that considers the context in which a word is used. In this case, the word "pentameter" strongly suggests that the correct word is "iambic", since "iambic pentameter" is a well-known poetic form, while "gambit pentameter" is not a commonly used term. The frequency-based method, on the other hand, may not have taken into account this contextual information and relied solely on the frequency of the individual words.

In summary, both perplexity-based and frequency-based corrections have their limitations and are not infallible. In this comparison, both methods are demonstrating comparable performance, as each method correctly identified 6 strictly correct candidates. However, the candidate replacements suggested by each method were not always the same, indicating that each method has its own strengths and weaknesses. The advantages of the perplexity-based method is that it is context-aware and better able to handle ambiguous cases. The perplexity-based method can often suggest a more appropriate replacement for a mistyped word when the correct one is not immediately clear, by considering the context of the sentence. This was evident in sentence 15, where the method correctly identified "iambic" as the most suitable candidate. Conversely, the frequency-based method is less influenced by noisy training data, as it does not consider the context of the sentence. This can lead to more accurate predictions when the misspelled word is a common misspelling of the correct word. This was beneficial in sentence 10, where the correctly spelled word "accommodate" was identified, highlighting the method's capability to avoid being affected by inaccuracies or inconsistencies in the training data. Despite their individual benefits, neither method is perfect, and there is room for improvement in selecting better candidate replacements. 


**What extra resources/information could we use to pick better candidate replacements?**

The following are some suggestions to enhance the correction process:

- *Use a combination of methods*: We could combine perplexity and frequency scores, or integrate other scoring methods, to create a more comprehensive ranking system. This can help leverage the strengths of each method and potentially reduce the impact of their individual weaknesses.

- *Leverage external resources*: We could employ external resources, such as dictionaries, thesauri, or domain-specific word lists, to provide additional context or refine the candidate selection process. We could also incorporate other sources of information such as syntactic and semantic constraints, as well as using a larger and more diverse training dataset.

- *Use ensemble methods*: We could employ multiple language models or algorithms to generate candidate replacements, and then select the best option based on a consensus or weighted vote among them.

- *Take into account keyboard layout*: we could consider key proximity when identifying candidate replacements for mistyped words. This approach is based on the idea that typos often occur due to accidentally pressing a nearby key instead of the intended one. To implement this strategy, we can utilize a keyboard distance metric, which calculates the physical distance between keys on a standard keyboard layout. We could compare the mistyped word with possible candidate words to determine which candidates have the smallest cumulative keyboard distance for all the differing characters. By considering keyboard distance as an additional factor, the correction process can be more effective in handling typos that arise from accidental key presses. An example of this is seen in sentence 3 with the mistyped word "grozn", where the correct word "groan" only differs by a single letter, and these two letters "z" and "a" are located adjacent to each other on the keyboard layout.

It is important to note that perplexity is an interplay between the trained language model and the test set. The performance of the perplexity-based method is contingent on the quality of the trained language model and the representativeness of the test set. If the test set differs significantly from the training data, the perplexity-based method may struggle to provide accurate candidate replacements, as the language model may not have encountered similar contexts during training.